In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [3]:
df=pd.read_csv("C:\\Users\\adity\\Downloads\\WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [6]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [7]:
df.drop('customerID', axis=1, inplace=True)

In [8]:
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})


In [9]:
from sklearn.impute import SimpleImputer
import pandas as pd

num_cols = df.select_dtypes(include=['int64', 'float64']).columns
num_cols = num_cols.drop('Churn')

imputer = SimpleImputer(strategy='median')

df[num_cols] = pd.DataFrame(
    imputer.fit_transform(df[num_cols]),
    columns=num_cols,
    index=df.index
)


In [10]:
cat_cols = df.select_dtypes(include='object').columns
le = LabelEncoder()
for col in cat_cols: df[col] = le.fit_transform(df[col])


In [11]:
X = df.drop('Churn', axis=1)
y = df['Churn']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [13]:
lr = LogisticRegression(max_iter=1000,class_weight='balanced',   random_state=42)
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_lr))


Accuracy: 0.7565649396735273


In [14]:
print("NaNs in X_train:", np.isnan(X_train).sum())
print("NaNs in X_test :", np.isnan(X_test).sum())


NaNs in X_train: 0
NaNs in X_test : 0


In [16]:
def encode_inputs(gender, contract, tenure, monthly_charges):
    gender_map = {'male': 1, 'female': 0}
    contract_map = {
        'month-to-month': 0,
        'one year': 1,
        'two year': 2
    }

    return {
        'gender': gender_map[gender],
        'SeniorCitizen': 0,
        'Partner': 0,
        'Dependents': 0,
        'tenure': tenure,
        'PhoneService': 1,
        'MultipleLines': 0,
        'InternetService': 2,
        'OnlineSecurity': 0,
        'OnlineBackup': 0,
        'DeviceProtection': 0,
        'TechSupport': 0,
        'StreamingTV': 1,
        'StreamingMovies': 1,
        'Contract': contract_map[contract],
        'PaperlessBilling': 1,
        'PaymentMethod': 2,
        'MonthlyCharges': monthly_charges,
        'TotalCharges': tenure * monthly_charges
    }


In [ ]:
gender = input("Gender (Male/Female): ").strip().lower()
contract = input("Contract (Month-to-month / One year / Two year): ").strip().lower()
tenure = int(input("Tenure (in months): "))
monthly_charges = float(input("Monthly Charges: "))

def explain_prediction(gender, contract, tenure, monthly_charges, churn_prob):
    reasons = []

    # Contract impact
    if contract == 'month-to-month':
        reasons.append("Short-term (month-to-month) contract increases churn risk")
    else:
        reasons.append("Long-term contract reduces churn risk")

    # Tenure impact
    if tenure < 12:
        reasons.append("Customer has low tenure (new customer)")
    else:
        reasons.append("Customer has high tenure (loyal customer)")

    # Charges impact
    if monthly_charges > 80:
        reasons.append("High monthly charges increase dissatisfaction")
    else:
        reasons.append("Affordable monthly charges reduce churn risk")

    # Final summary
    if churn_prob > 0.5:
        summary = "Overall risk factors outweigh retention factors"
    else:
        summary = "Retention factors outweigh churn risk factors"

    return reasons, summary

input_data = encode_inputs(gender, contract, tenure, monthly_charges)
input_df = pd.DataFrame([input_data])

input_df[num_cols] = imputer.transform(input_df[num_cols])
input_scaled = scaler.transform(input_df)
prediction = lr.predict(input_scaled)
probability = lr.predict_proba(input_scaled)[0][1]
reasons, summary = explain_prediction(
    gender,
    contract,
    tenure,
    monthly_charges,
    probability
)

print("\nPrediction Result:")

if prediction[0] == 1:
    print("🚨 Customer is likely to CHURN")
else:
    print("✅ Customer is likely to STAY")

print(f"Churn Probability: {probability*100:.2f}%\n")

print("📌 Reason:")
for r in reasons:
    print(" -", r)

print("➡", summary)



In [67]:
import joblib

joblib.dump(lr, "churn_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(imputer, "imputer.pkl")
joblib.dump(num_cols, "num_cols.pkl")


['num_cols.pkl']